### Differential Evolution on Gas Transmission Pipe

### DE/current to best/1

In [1]:
# Import library yang dibutuhkan
import numpy as np

### Differential evolution method

In [ ]:
# Fungsi untuk Mutation
# Input : target pipe dan F
# Output : mutated vector
def mutation(x, F):
    return np.add(x[0], np.add(np.multiply(F, np.subtract(x[1], x[2])), np.multiply(F, np.subtract(x[3], x[4]))))

In [3]:
# Fungsi untuk Crossover
# Input : mutated vector, target pipe, dimensi dari pipe = 7, dan CR
# Output : trial vector
def crossover(mutated, target, dims, cr):
    # Generate uniform random value untuk setiap dimension
    p = np.random.rand(dims)
    # Generate Trial Vector dari binomial crossover
    trial = [mutated[i] if p[i] < cr else target[i] for i in range(dims)]
    return trial

### Methods

In [4]:
# Fungsi untuk mengecek batasan dari pd, ps, L, D pada tiap pipe dari 1 set pipe
# Input : mutated pipe dan batasan
# Output : mutated pipe yang memenuhi batasan
def check_bounds(mutated, bounds):
    mutated_bound = []
    for i in range(len(mutated)):
        temp = []
        for j in range(len(bounds)):
            temp.append(np.clip(mutated[i][j], bounds[j, 0], bounds[j, -1]))
        mutated_bound.append(temp)
    return mutated_bound

In [5]:
# Fungsi untuk cek constraint pertidaksamaan
# 1.) Pd/Ps >= 1
# 2.) Pd/Ps <= Ki
# 3.) Flow rate kuadrat = 0
# Input : target pipe
# Output : pipe yang memenuhi constraint
def check_inequality(pop, K=[]):
	# Cek Pd >= Ps
	for p in pop:
		if p[0] <= p[1]:
			p[0],p[1] = p[1],p[0]
	# Cek K
	# Cek Q
	return pop
def check_inequality_2(pop):
	ps_init = 500
	for i in range(len(pop)):
		if i == 0:
			if (pop[i][0]/ps_init) < 1:
				return False
		else:
			if (pop[i][0]/pop[i-1][1]) < 1:
				return False
	return True

In [6]:
# Fungsi untuk cek total panjang dari branch 1 dan branch 2
def check_length1(pop):
    return True if round(sum(pop[:,2])) == 175 else False
# Fungsi untuk cek total panjang dari branch 1 dan branch 3
def check_length2(pop):
    return True if round(sum(pop[:,2])) == 200 else False

### Process

Bentuk dari populasi awal untuk setiap pipe:</br>
pipe1 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
pipe2 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
pipe3 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
...</br>
pipe11 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
  
Setelah diolah DE, output yang diharapkan yaitu vector paling optimal:</br>
pipe1 = [pd, ps, l, d]</br>
pipe2 = [pd, ps, l, d]</br>
pipe3 = [pd, ps, l, d]</br>
...</br>
pipe11 = [pd, ps, l, d]</br>

In [7]:
# Definisi parameter untuk Differential Evolution
# ------------------------------
pop_size = 2000 # size population candidate solution
iter = 100  # number iteration

F = 0.02     # scale factor [0, 1]
CR = 0.8    # crossover rate [0, 2]

n = 10  # number compressor
m = n+1 # number pipe
N1 = 4  # number compressor in branch 1
N2 = 3  # number compressor in branch 2
N3 = 3  # number compressor in branch 3

In [8]:
# Definisi batas atas dan batas bawah untuk setiap dimensi pipe
# pipe = [pd, ps, l, d]

# Bounds for branch 1
bounds1 = np.asarray([
    (590, 1000),(300, 900),(2, 70),(4, 36)
])
# Bounds for branch 2
bounds2 = np.asarray([
    (590, 1000),(300, 900),(2, 30),(4, 18)
])
# Bounds for branch 3
bounds3 = np.asarray([
    (590, 1000),(200, 860),(2, 70),(4, 18)
])

In [9]:
# Fungsi Q untuk menghitung Flow Rate
# Input : pd, ps, L, D
# Output : Q
def q(pd, ps, l, d):
    return (871 * ((d)**(8/3))) * np.sqrt((pd**2 - ps**2) / l)

In [10]:
# Objective Function F()
# x = [pd, ps, l, d]

# Fungsi objektif untuk operating cost compressor
def f_comp(x, q, co = 8, cc = 70.00, T = 520, k = 1.26, z = 0.9):
  return ((co + cc) * q * 0.08531 * T * (k/(k - 1)) * ((x[0]/x[1])**((z*(k - 1))/k) - 1))
# Fungsi objektif untuk maintenance cost pipe
def f_pipe(x, cs = 870):
  return (cs * x[2] * x[3])

# Fungsi objektif
# Input: 1 set pipe (pipe1, pipe2, ..., pipei), jumlah compressor, jumlah pipe
# Output: sum fungsi objektif
def f(pop, n, m):
    sum = 0
    for i in range(n):
      qi = q(pop[i][0],pop[i][1],pop[i][2],pop[i][3])/1000000
      sum += f_comp(pop[i], qi)
      for j in range(m):
          sum += f_pipe(pop[j])
    return sum

#### Branch 1 + Branch 2

In [11]:
# Fungsi untuk mengolah populasi awal pipa dari branch 1 dan branch 2 yang hanya memenuhi panjang total
# Input : populasi awal pipe branch 1 + branch 2, ukuran populasi, N1, dan N2
# Output : vector populasi awal yang memenuhi panjang dari branch 1 dan branch 2
def get_pipe_length1(pipe_b1_b2, pop_size, N1, N2):
    best_pipe = []
    for i in range(pop_size): # 1, 2, ..., 1000
        temp = []
        sum_pipe = 0
        for j in range(N1+N2): # 1, 2, 3, 4, 5, 6, 7
            sum_pipe += pipe_b1_b2[j][i][2]
            temp.append(pipe_b1_b2[j][i])
        if(round(sum_pipe) == 175):
            best_pipe.append(temp)
    return best_pipe

In [12]:
# Proses inisialisasi populasi awal setiap pipe pada branch 1 dan branch 2
# <-----Branch 1-----> <--------Branch 2--------->
# pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7,

while True:
  # Initialize population pipe branch 1
  pipe_b1 = []
  for i in range(N1-1):
    pipe_b1.append(bounds1[:, 0] + (np.random.rand(pop_size, len(bounds1)) * (bounds1[:, 1] - bounds1[:, 0])))
  # Initialize population pipe branch 2
  pipe_b2 = []
  for i in range(N2+1):
    pipe_b2.append(bounds2[:, 0] + (np.random.rand(pop_size, len(bounds2)) * (bounds2[:, 1] - bounds2[:, 0])))
  # Concatenate population pipe branch 1 and branch 2
  pipe_b1_b2 = np.concatenate((pipe_b1, pipe_b2))
  # Get all pipes whichs meet constraint length branch 1 and branch 2
  best_pipe_b1_b2 = get_pipe_length1(pipe_b1_b2, pop_size, N1, N2)
  if(best_pipe_b1_b2 != []):
    break

In [13]:
# Cek berapa populasi awal vector yang memenuhi panjang total
len(best_pipe_b1_b2)

18

In [14]:
# Fungsi differential evolution untuk pipe branch 1 dan branch 2
def diff_evol1(pop_pipe, iter, pop_size, bounds, F, CR):

    # Check inequality
    pop_pipe = [check_inequality(p) for p in pop_pipe]
    pop_pipe = np.array(pop_pipe)

    # Evaluate initial population candidate solution
    obj_all = [f(p, N1+N2, N1+N2) for p in pop_pipe]

    # Find the best vector of initial population
    best_vector = pop_pipe[np.argmin(obj_all)]
    best_obj = min(obj_all)
    prev_obj = best_obj

    for i in range(iter):
        # Iterate over all candidate solution
        for j in range(pop_size):
            # MUTATION
            while(True):
                # Choose 3 candidate solution : a, b, c, d, e.
                a = pop_pipe[j] # current
                b = pop_pipe[np.argmin(obj_all)] # best
                c = pop_pipe[j] # current
                # Choose 5 candidate solution : a, b, c, d, e
                candidates = [candidate for candidate in range(pop_size) if candidate != j]
                d, e = pop_pipe[np.random.choice(candidates, 2, replace=False)]
                # Perform mutation
                mutated = mutation([a, b, c, d, e], F)
                # Check bound mutated vector
                mutated_b1 = check_bounds(mutated[:3], bounds[0])
                mutated_b2 = check_bounds(mutated[3:], bounds[1])
                mutated = np.concatenate((mutated_b1, mutated_b2))
                # Check inequality const
                mutated = check_inequality(mutated)
                    # Check lenght, if true continue, else LOOP UNTIL
                if check_length1(mutated):
                    break

            # CROSSOVER
            # Perform crossover
            while(True):
                trial = np.array(crossover(mutated, pop_pipe[j], N1+N2, CR))
                if check_length1(trial):
                    break

            # Compute objective function value for target vector
            obj_target = f(pop_pipe[j], N1+N2, N1+N2)
            # Compute objective function value for trial vector
            obj_trial = f(trial, N1+N2, N1+N2)

            # SELECTION
            # Perform selection
            if obj_trial < obj_target:
               # Replace target vector with trial vector
               pop_pipe[j] = trial
               # Store the new obj function value
               obj_all[j] = obj_trial

        # Find best performing vector each iteration
        best_obj = min(obj_all)
        # Store the lowest obj function value
        if best_obj < prev_obj:
            best_vector = pop_pipe[np.argmin(obj_all)]
            prev_obj = best_obj

        # Print progress iteration
        # print('Iteration %d : f[%s] = %.5f' % (i, np.around(best_vector, decimals=5), best_obj))
        # print('--------------------')

    return [best_vector, best_obj]

In [15]:
# Proses Differential Evolution hingga memenuhi setiap constraint
# Hingga memenuhi Ps demand 1 = 600 psi
while(True):
    best_b1_b2 = diff_evol1(best_pipe_b1_b2, iter, len(best_pipe_b1_b2), [bounds1, bounds2], F, CR)
    status = False
    print(best_b1_b2)
    if(round(best_b1_b2[0][N1+N2-1][1]) == 600 and check_inequality_2(best_b1_b2[0])):
        break

[array([[593.56324182, 426.19708678,  18.09884584,   6.58107426],
       [784.56124641, 669.15851756,  32.6124657 ,   6.67332948],
       [709.19771401, 361.09606293,  59.1813526 ,   4.        ],
       [898.49118081, 465.26222321,  10.18266695,   6.85474661],
       [735.97283605, 587.46972248,  13.77341136,   7.07036986],
       [932.38359437, 690.38216767,  28.91347618,  15.91913035],
       [801.22636551, 564.11346831,  12.01799864,   4.45570436]]), np.float64(7921282.4253472155)]
[array([[590.01641453, 447.61785437,  15.52470777,   7.937087  ],
       [781.49629609, 691.12976613,  29.59228116,   9.5688057 ],
       [706.4606435 , 375.69440426,  62.52497344,   4.        ],
       [883.45819948, 485.19927665,  10.66028705,   6.61743703],
       [734.70901178, 591.81397078,  13.6491417 ,   7.56036536],
       [928.9686183 , 698.86507908,  29.77312429,  16.11764342],
       [804.98857799, 562.76682109,  12.89363731,   4.47482673]]), np.float64(8595196.425966324)]
[array([[590.        

In [16]:
# Vector terbaik untuk branch 1 dan branch 2
best_b1_b2

[array([[595.72017739, 428.77159608,  17.2938745 ,   6.00658485],
        [786.8085487 , 706.43911678,  32.96406659,   6.50831657],
        [735.5174675 , 366.90072713,  58.31049587,   4.32571185],
        [859.27893538, 453.13599929,  10.80427704,   7.23196894],
        [732.51348532, 606.57153433,  14.98644702,   7.59315238],
        [917.85790099, 699.85933993,  29.15956044,  15.741617  ],
        [833.5554837 , 600.23625531,  11.73243811,   5.33057726]]),
 np.float64(8077250.287144778)]

#### Branch 1 + Branch 3

In [17]:
# Mengambil vector pipe pada branch 1 yang akan digunakan untuk evaluasi
demand1_index = 0
for i in range(len(best_b1_b2[0])):
    if round(best_b1_b2[0][i][1]) == 600:
        demand1_index = i
best_b1 = []
for p in best_b1_b2[0]:
    if(round(p[1]) != 600):
        best_b1.append(p)
    if len(best_b1) == N1-1:
        break

In [18]:
# Fungsi untuk mengolah populasi awal pipa dari branch 1 dan branch 3 yang hanya memenuhi panjang total
# Input : populasi awal pipe branch 1 + branch 3, ukuran populasi, N1, dan N3
# Output : vector populasi awal yang memenuhi panjang dari branch 1 dan branch 3
def get_pipe_length2(pipe_b3, best_b1, pop_size, N1, N3):
    best_pipe = []
    for i in range(pop_size): # 1, 2, ..., 50
        temp = []
        sum_pipe = 0
        for j in range(N1+N3): # 1, 2, 3, 4, 5, 6, 7
            if j < N1-1: # 0, 1, 2
                sum_pipe += best_b1[j][2]
            if j >= N1-1: # 3, 4, 5, 6 => 0, 1, 2, 3
                sum_pipe += pipe_b3[j-N3][i][2]
                temp.append(pipe_b3[j-N3][i])
        if(round(sum_pipe) == 200):
            best_pipe.append(temp)
    return best_pipe

In [19]:
# Proses inisialisasi populasi awal setiap pipe pada branch 3
# <-----Branch 1-----> <--------Branch 3----------->
# pipe1, pipe2, pipe3, pipe8, pipe9, pipe10, pipe11,
def create_pipe_b3():
  while True:
    # Initialize population pipe branch 3
    pipe_b3 = []
    for i in range(N3+1):
      pipe_b3.append(bounds3[:, 0] + (np.random.rand(pop_size, len(bounds3)) * (bounds3[:, 1] - bounds3[:, 0])))
    best_pipe_b3 = get_pipe_length2(pipe_b3, best_b1, pop_size, N1, N3)
    if(best_pipe_b3 != [] and len(best_pipe_b3) > 4):
      break
  return best_pipe_b3

In [20]:
def diff_evol_2(pop_pipe_b3, best_b1, iter, pop_size, bounds, F, CR):

    # Check inequality branch 3
    pop_pipe = [check_inequality(p) for p in pop_pipe_b3]
    pop_pipe = np.array(pop_pipe)

    # Evaluate initial population candidate solution
    temp = [np.concatenate((best_b1, p)) for p in pop_pipe_b3]
    obj_all = [f(p, N1+N3, N1+N3) for p in temp]

    # Find the best vector of initial population
    best_vector = pop_pipe[np.argmin(obj_all)]
    best_obj = min(obj_all)
    prev_obj = best_obj

    for i in range(iter):
        # Iterate over all candidate solution
        for j in range(pop_size):
            # MUTATION
            while(True):
                # Choose 3 candidate solution : a, b, c.
                a = pop_pipe[j] # current
                b = pop_pipe[np.argmin(obj_all)] # best
                c = pop_pipe[j] # current
                # Choose 5 candidate solution : a, b, c, d, e
                candidates = [candidate for candidate in range(pop_size) if candidate != j]
                d, e = pop_pipe[np.random.choice(candidates, 2, replace=False)]
                # Perform mutation
                mutated = mutation([a, b, c, d, e], F)
                # Check bound mutated vector
                mutated = check_bounds(mutated, bounds)
                # Check inequality const
                mutated = check_inequality(mutated)
                # Check lenght, if true continue, else LOOP UNTIL
                temp = np.concatenate((best_b1, mutated))
                if check_length2(temp):
                    break
            # CROSSOVER
            # Perform crossover
            while(True):
                mutated_temp = np.concatenate((best_b1, mutated))
                target_temp = np.concatenate((best_b1, pop_pipe[j]))
                trial = np.array(crossover(mutated_temp, target_temp, N1+N3, CR))
                if check_length2(trial):
                    break

            # Compute objective function value for target vector]
            obj_target = f(target_temp, N1+N3, N1+N3)
            # Compute objective function value for trial vector
            obj_trial = f(trial, N1+N3, N1+N3)

            # SELECTION
            # Perform selection
            if obj_trial < obj_target:
               # Replace target vector with trial vector
               pop_pipe[j] = trial[3:]
               # Store the new obj function value
               obj_all[j] = obj_trial

        # Find best performing vector each iteration
        best_obj = min(obj_all)
        # Store the lowest obj function value
        if best_obj < prev_obj:
            best_vector = pop_pipe[np.argmin(obj_all)]
            prev_obj = best_obj

    #     # Print progress iteration
    #     # print('Iteration %d : f[%s] = %.5f' % (i, np.around(best_vector, decimals=5), best_obj))
    #     # print('--------------------')
    best_vector = np.concatenate((best_b1, best_vector))
    return [best_vector, best_obj]

In [21]:
while(True):
    best_pipe_b3 = create_pipe_b3()
    best_b1_b3 = diff_evol_2(best_pipe_b3, best_b1, iter, len(best_pipe_b3), bounds3, F, CR)
    status = False
    if(round(best_b1_b3[0][N1+N3-1][1]) == 300):
        status = True
    if status and (best_b1_b3[0][:3] == best_b1).all() and check_inequality_2(best_b1_b3[0]):
        break

In [22]:
best_b1_b3

[array([[595.72017739, 428.77159608,  17.2938745 ,   6.00658485],
        [786.8085487 , 706.43911678,  32.96406659,   6.50831657],
        [735.5174675 , 366.90072713,  58.31049587,   4.32571185],
        [746.96487508, 521.51988056,   6.45540415,   7.99102708],
        [715.64843365, 442.40860122,  15.43936647,  12.35979458],
        [857.32794097, 604.45232054,  24.14699842,   4.05610056],
        [796.25249107, 300.42022757,  44.93887503,   4.67600337]]),
 np.float64(7087874.146098909)]

### Results

In [23]:
# Best vector untuk pipe di branch 1
print('Branch 1')
for i in range(len(best_b1)):
    print('Pipe {0}: {1}'.format(i+1, best_b1[i]))


Branch 1
Pipe 1: [595.72017739 428.77159608  17.2938745    6.00658485]
Pipe 2: [786.8085487  706.43911678  32.96406659   6.50831657]
Pipe 3: [735.5174675  366.90072713  58.31049587   4.32571185]


In [24]:
# Best vector untuk pipe di branch 1 dan branch 2
print('Branch 1 dan Branch 2')
for i in range(len(best_b1_b2[0])):
    print('Pipe {0}: {1}'.format(i+1, best_b1_b2[0][i]))
    if round(best_b1_b2[0][i][1]) == 600:ps1 = i
print('--------------------------------')
print('F(x) = ',best_b1_b2[1])
print('Sum L = ',sum(best_b1_b2[0][:,2]))

print('Ps == 600 terpenuhi pada pipe ke',ps1+1)


Branch 1 dan Branch 2
Pipe 1: [595.72017739 428.77159608  17.2938745    6.00658485]
Pipe 2: [786.8085487  706.43911678  32.96406659   6.50831657]
Pipe 3: [735.5174675  366.90072713  58.31049587   4.32571185]
Pipe 4: [859.27893538 453.13599929  10.80427704   7.23196894]
Pipe 5: [732.51348532 606.57153433  14.98644702   7.59315238]
Pipe 6: [917.85790099 699.85933993  29.15956044  15.741617  ]
Pipe 7: [833.5554837  600.23625531  11.73243811   5.33057726]
--------------------------------
F(x) =  8077250.287144778
Sum L =  175.2511595621332
Ps == 600 terpenuhi pada pipe ke 7


In [25]:
# Best vector untuk pipe di branch 1 dan branch 3
print('Branch 1 dan Branch 3')
for i in range(len(best_b1_b3[0])):
    j = i+4 if i > 2 else i
    if round(best_b1_b3[0][i][1]) == 300:ps2 = j
    print('Pipe {0}: {1}'.format(j+1, best_b1_b3[0][i]))
print('--------------------------------')
print('F(x) = ',best_b1_b3[1])
print('Sum L = ',sum(best_b1_b3[0][:,2]))

print('Ps == 300 terpenuhi pada pipe ke',ps2+1)

Branch 1 dan Branch 3
Pipe 1: [595.72017739 428.77159608  17.2938745    6.00658485]
Pipe 2: [786.8085487  706.43911678  32.96406659   6.50831657]
Pipe 3: [735.5174675  366.90072713  58.31049587   4.32571185]
Pipe 8: [746.96487508 521.51988056   6.45540415   7.99102708]
Pipe 9: [715.64843365 442.40860122  15.43936647  12.35979458]
Pipe 10: [857.32794097 604.45232054  24.14699842   4.05610056]
Pipe 11: [796.25249107 300.42022757  44.93887503   4.67600337]
--------------------------------
F(x) =  7087874.146098909
Sum L =  199.54908101799217
Ps == 300 terpenuhi pada pipe ke 11


In [26]:
# Best vector untuk pipe di branch 1, branch 2, dan branch 3
best_all = np.concatenate((best_b1_b2[0],best_b1_b3[0][3:]))
print('Best vector setiap branch:')
print('\t|     Pd     |     Ps     |     L       |      D      |        Q         |')
for i in range(11):
    print('Pipe {0}: {1}'.format(i+1, best_all[i]),end="")
    if round(best_all[i][1]) == 600:ps1 = i
    if round(best_all[i][1]) == 300:ps2 = i
    print(' ',q(best_all[i][0],best_all[i][1],best_all[i][2],best_all[i][3])/1000000)
    if i == 2 or i == 6 or i == 10:
        print('---------------------------------')
print('Ps = 600 terpenuhi pada pipe ke:',ps1+1)
print('Ps = 300 terpenuhi pada pipe ke:',ps2+1)
print('---------------------------------')
print('F(x) = {0}'.format(f(best_all, n, m)))

Best vector setiap branch:
	|     Pd     |     Ps     |     L       |      D      |        Q         |
Pipe 1: [595.72017739 428.77159608  17.2938745    6.00658485]  10.326613379624433
Pipe 2: [786.8085487  706.43911678  32.96406659   6.50831657]  7.759949954731851
Pipe 3: [735.5174675  366.90072713  58.31049587   4.32571185]  3.6120941715577106
---------------------------------
Pipe 4: [859.27893538 453.13599929  10.80427704   7.23196894]  37.839565291825956
Pipe 5: [732.51348532 606.57153433  14.98644702   7.59315238]  20.580238611994055
Pipe 6: [917.85790099 699.85933993  29.15956044  15.741617  ]  149.08617302431426
Pipe 7: [833.5554837  600.23625531  11.73243811   5.33057726]  12.752841656774988
---------------------------------
Pipe 8: [746.96487508 521.51988056   6.45540415   7.99102708]  46.790709978565936
Pipe 9: [715.64843365 442.40860122  15.43936647  12.35979458]  101.82874804532038
Pipe 10: [857.32794097 604.45232054  24.14699842   4.05610056]  4.5092502784554025
Pipe 11: 

In [27]:
# TODO 1. Hitung K
Ps_init = 500.00
K = []

print('Compressor Ratio')
for i in range(len(best_all)-1):
	if i == 0: K.append(best_all[i][0]/Ps_init)
	else:
		K.append(best_all[i][0]/best_all[i-1][1])
	print('Compressor {0} : {1}'.format(i+1,K[i]))

Compressor Ratio
Compressor 1 : 1.1914403547758814
Compressor 2 : 1.8350295493033855
Compressor 3 : 1.0411618638280784
Compressor 4 : 2.341993002042445
Compressor 5 : 1.6165422444202353
Compressor 6 : 1.5131898696768564
Compressor 7 : 1.1910328778067063
Compressor 8 : 1.2444514447081696
Compressor 9 : 1.3722361511666583
Compressor 10 : 1.937864541074533
